## 이 출처 따라서 yolov3 만들어보기
https://wolfy.tistory.com/259 <br>
https://csm-kr.tistory.com/11

- coco형식 : (x,y,w,h) --> (좌상단 x, 좌상단 y, bouding box의 W, bounding box의 H)
- YOLO형식 : (x,y,w,h) --> (bounding box 중심점의 x,bounding box 중심점의 y,bouding box의 W, bounding box의 H)

In [4]:
'''
data = [['a.png', 'AC', 376, 108, 289, 274, 376, 108, 665, 382],
        ['b.png', 'AC', 317, 130, 296, 254, 317, 130, 613, 384],
      ['c.png', 'HC', 245, 62, 408, 325, 245, 62, 653, 387],
        ['d.png', 'FL', 397, 194, 168, 165, 397, 194, 565, 359],
      ['e.png', 'HUM', 367, 121, 311, 283, 367, 121, 678, 404]]
df = pd.DataFrame(data, columns=['img_name', 'class', 'X', 'Y', 'W', 'H', 'X_min', 'Y_min', 'X_max', 'Y_max'])
df.to_csv("./tmp.csv", index=False)
'''

출처 : https://yeko90.tistory.com/entry/opencv-yolov3-%EC%82%AC%EC%9A%A9-%EB%B0%A9%EB%B2%95
<br>
바운딩 박스 크기 다시 구하기

In [1]:
import pandas as pd

In [2]:
d = pd.read_csv('./tmp.csv')

In [3]:
d

,img_name,class,X,Y,W,H,X_min,Y_min,X_max,Y_max
0,a.png,AC,376,108,289,274,376,108,665,382
1,b.png,AC,317,130,296,254,317,130,613,384
2,c.png,HC,245,62,408,325,245,62,653,387
3,d.png,FL,397,194,168,165,397,194,565,359
4,e.png,HUM,367,121,311,283,367,121,678,404


In [47]:
x = d['X']
y = d['Y']
w = d['W']
h = d['H']
bbox = []

for i in range(5):
    a = (x[i], y[i], w[i], h[i])
    bbox.append(a)

In [48]:
for a in bbox:
    print(a)

(376, 108, 289, 274)
(317, 130, 296, 254)
(245, 62, 408, 325)
(397, 194, 168, 165)
(367, 121, 311, 283)


In [49]:
bbox[4][0]

367

# Pascal VOC 2012 Dataset 탐색
출처 : https://velog.io/@tataki26/open-cv

In [1]:
import sys

from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler

from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## utils
import numpy as np
import pandas as pd
import random, math, time
from tqdm.notebook import tqdm

## File Loader
import os#, xmltodict
import os.path as pth
from PIL import Image

# Draw Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## Transformer
from random import sample
import albumentations as A
from albumentations.pytorch.transforms import ToTensor

# Seed
random.seed(53)

ModuleNotFoundError: No module named 'torch'

In [ ]:
# configs
root_dir = '/content'
annot_f = './{}/VOCdevkit/VOC2007/Annotations'
image_f = './{}/VOCdevkit/VOC2007/JPEGImages/{}'

classes = ['person', # Person
           'bird', 'cat', 'cow', 'dog', 'horse', 'sheep', # Animal
           'aeroplane', 'bicycle', 'boat', 'bus', 'car', 'motorbike', 'train', # Vehicle
           'bottle', 'chair', 'dining table', 'potted plant', 'sofa', 'tv/monitor' # Indoor
           ]

num_classes = len(classes)
feature_size = 7
num_bboxes = 2

In [81]:
# 수정
path = 'C:/Users/JungHyeona/Documents/deeplearning_seminar/Biometry_20200722'

csv_path = './train/info/'
img_path = './train/train_img'
image_f = path + '/train/train_img/{}/{}'

classes = ['AC', 'FL', 'HC', 'HUM']

num_classes = len(classes)
feature_size = 7
num_bboxes = 2

## Dataframe

In [12]:
import pandas as pd

In [89]:
x_y = pd.read_csv('./train/info/img_info(x,y,w,h).csv')
minmax = pd.read_csv('./train/info/labelCoordinate_minmax.csv')

In [91]:
x_y.head()

,image_name,class_name,X,Y,W,H
0,20160307_E0000776_I0068889.png,AC,376,108,289,274
1,20160307_E0000776_I0068893.png,AC,317,130,296,254
2,20160307_E0000777_I0069006.png,AC,245,62,408,325
3,20160307_E0000777_I0069020.png,AC,397,194,168,165
4,20160307_E0000778_I0069096.png,AC,367,121,311,283


In [93]:
minmax.head()

,image_name,X_min,Y_min,X_max,Y_max
0,20160307_E0000776_I0068889.png,376,108,665,382
1,20160307_E0000776_I0068893.png,317,130,613,384
2,20160307_E0000777_I0069006.png,245,62,653,387
3,20160307_E0000777_I0069020.png,397,194,565,359
4,20160307_E0000778_I0069096.png,367,121,678,404


In [51]:
d['img_name'][0]

'a.png'

## utils

In [4]:
def draw_image(d, w=320, h=320, transforms=None):
    im = np.array(Image.open(image_f.format('AC', minmax['image_name'][0])).resize((w,h)), dtype=np.uint8)

    # Create figure and axes
    fig, ax = plt.subplots(1, figsize=(7,7))

    xmin = d['X_min']
    xmax = d['X_max']
    ymin = d['Y_min']
    ymax = d['Y_max']
    la = d['class_name']

    if transforms:
        sample = transforms(image=im, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax, category_ids=la)
        im = sample['image'].permute(1,2,0).numpy()
        la = sample['category_ids']

    # Display the image
    ax.imshow(im)


    # Create a Rectangle patch
    # top left (x, y) , (w, h)
    rect = patches.Rectangle((d['X'], d['Y']), d['W'], d['H'], linewidth=1, edgecolor='r', facecolor='none')
    
    # Add the patch to the Axes
    ax.add_patch(rect)
    props = dict(boxstyle='round', facecolor='red', alpha=0.9)
    plt.text(d['X'], d['Y'], classes[0], fontsize=10, color='white', bbox=porps)
    plt.axis('off')
    plt.show()

In [5]:
draw_image(d)

NameError: name 'np' is not defined

## image와 box, label정보를 가져와주는 함수

기존 Bounding Box 좌표 값인 (x1, x2, y1, y2) 형식을
Yolo Style인 이미지 크기에 대한 비율 값으로 바꾸고, (centerX, centerY, w, h) 형식으로 바꾸는 소스코드이다.
<br><br>
출처
- https://eehoeskrap.tistory.com/357
- https://cjsal95.tistory.com/8
- 사용 : https://machine-does-not-lie.tistory.com/entry/min-max-%EC%A2%8C%ED%91%9C-YOLO-label-format-%EB%B3%80%ED%99%98

In [45]:
result = []
label = []
bbox = []
i_width=421
i_height=587

def get_object_params(xmin, ymin, xmax, ymax):
    image_width = 1.0 * i_width
    image_height = 1.0 * i_height
 
    center_x = xmin + 0.5 * (xmax - xmin)
    cneter_y = ymin + 0.5 * (ymax - ymin)
    
    absolute_width = xmax - xmin
    absolute_height = ymax - ymin
    
    # (centerX, centerY, w, h) 형식으로 바꿈
    l_x = round(center_x / image_width , 6)
    l_y = round(cneter_y / image_height , 6)
    l_width = round(absolute_width / image_width , 6)
    l_height = round(absolute_height / image_height , 6)
    
    # 이미지 이름 및 label
    img_name = d['img_name']
    #image_size = np.asarray(tuple(map(int, info['size'].values()))[:2], np.int16)  # w, h
    label.append(d['class'])
    bbox = (l_x, l_y, l_width, l_height)
    
    result.append({'image_id':img_name, 'bboxs':bbox, 'labels':label})
    #result.append({'image_id':img_name, 'image_size':image_size, 'bboxs':bbox, 'labels':label})
    
    return result
    
# convert
# center_x : (Ditext X + 박스W / 2) / 원본이미지 W
# center_y : (Ditext Y + 박스H / 2) / 원본이미지 H
# box_width : 박스 W / 원본이미지 W
# box_height : 박스 H / 원본이미지 H

In [50]:
center_x = []
for i in range(len(d)):
    cx = get_object_params(bbox[i][0], bbox[i][1], bbox[i][2], bbox[i][3])
    center_x.append(cx)

print(center_x)

[[{'image_id': 0    a.png
1    b.png
2    c.png
3    d.png
4    e.png
Name: img_name, dtype: object, 'bboxs': (0.789786, 0.325383, -0.206651, 0.282794), 'labels': [0     AC
1     AC
2     HC
3     FL
4    HUM
Name: class, dtype: object, 0     AC
1     AC
2     HC
3     FL
4    HUM
Name: class, dtype: object, 0     AC
1     AC
2     HC
3     FL
4    HUM
Name: class, dtype: object, 0     AC
1     AC
2     HC
3     FL
4    HUM
Name: class, dtype: object, 0     AC
1     AC
2     HC
3     FL
4    HUM
Name: class, dtype: object]}, {'image_id': 0    a.png
1    b.png
2    c.png
3    d.png
4    e.png
Name: img_name, dtype: object, 'bboxs': (0.728029, 0.327087, -0.049881, 0.211244), 'labels': [0     AC
1     AC
2     HC
3     FL
4    HUM
Name: class, dtype: object, 0     AC
1     AC
2     HC
3     FL
4    HUM
Name: class, dtype: object, 0     AC
1     AC
2     HC
3     FL
4    HUM
Name: class, dtype: object, 0     AC
1     AC
2     HC
3     FL
4    HUM
Name: class, dtype: object, 0     AC
1     

In [36]:
bbox[0][0]

IndexError: list index out of range

In [56]:
center_x, center_y = [], []
width, height = [], []
for i in range(len(bbox)):
    cx, cy, w, h = get_object_params(bbox[i][0], bbox[i][1], bbox[i][2], bbox[i][3])
    center_x.append(cx)
    center_y.append(cy)
    width.append(w)
    height.append(h)

print(center_x)
print(center_y)
print(width)
print(height)

AttributeError: 'list' object has no attribute 'shape'

## train, vali 분리

In [ ]:
# utils
def get_tv_idx(tl, k = 0.7):
    
    total_idx = range(tl)
    train_idx = sample(total_idx, int(tl*k))  # sample = 라이브러리
    valid_idx = set(total_idx) - set(train_idx)
    
    return train_idx, list(valid_idx)

In [ ]:
# trval_list = get_infos()
train_idx, valid_idx = get_tv_idx(len(trval_list))

trval_list = np.asarray(trval_list)
train_list = trval_list[train_idx]
valid_list = trval_list[valid_idx]

## Dataset 정의

In [ ]:
# 예시
# trval_list = get_infos()
# VOCDataset(train_list, transforms=get_train_transforms())

class VOCDataset(Dataset):
    
    def __init__(self, data_list, mode='train', transforms=None):
        self.data_list = data_list
        self.mode = mode
        self.transforms = transforms

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        record = self.data_list[idx]
        img_id = record['image_id'] # 이미지 파일 이름
        bboxs = record['bboxs']     # x_min, x_max, y_min, y_max
        labels = record['labels']   # class

        img = Image.open(image_f.format(self.mode, img_id)) #.resize((800,800))
        img = np.array(img)

        if self.transforms:
            for t in self.transforms:
                sample = self.transforms(image=img, bboxes=bboxs, category_ids=labels)
                image = sample['image']
                bboxs = np.asarray(sample['bboxes'])
                labels = np.asarray(sample['category_ids'])  # labels = 이미지 폴더명


        if self.mode=='train':
            target = encode(bboxs, labels)
            return image, target
        
        else:
            return image

## bboxs(x,y,w,h) target으로 변환

In [ ]:
# Make YoLo Target
def encode(bboxs, labels):

    S = feature_size
    B = num_bboxes
    N = 5 * B + num_classes
    cell_size = 1.0 / float(S)
    # print(bboxs.shape)

    box_cxy = (bboxs[:, 2:] + bboxs[:, :2])/2.0
    box_wh = bboxs[:, 2:] - bboxs[:, :2]
    target = np.zeros((S, S, N))
    
    
    for b in range(bboxs.shape[0]):
        cxy, wh, label = box_cxy[b], box_wh[b], labels[b]
        ij = np.ceil(cxy / cell_size) - 1.0
        i, j = map(int, ij)
        top_left = ij*cell_size
        dxy_norm = (cxy-top_left)/cell_size
        
        for k in range(B):
            target[j, i, 5*k: 5*(k+1)] = np.r_[dxy_norm, wh, 1]
        target[j, i, 5*B+label] = 1.0
        
    return target

# Transform
- transform 내가 원래 쓰던걸로 해도 될지 고민해보기

In [ ]:
def get_train_transforms():
    return A.Compose([
        A.Resize(320, 320, always_apply=True, p=1),
        ToTensor(),
    ], bbox_params=A.BboxParams(format='albumentations', label_fields=['category_ids']))

## DataLoader

In [ ]:
# trval_list = get_infos()  =>  Data 불러옴.
train_ds = VOCDataset(train_list, transforms=get_train_transforms())
valid_ds = VOCDataset(valid_list, transforms=get_test_transforms())

# torch tensor를 batch size만큼 묶어줌
def collate_fn(batch):
    images, targets = zip(*batch)
    # 이미지 흑백이기에 1로 변환. 현재 이미지가 3(rgb)차원으로 되어있음.
    return torch.cat([img.reshape(-1, 1, 320, 320) for img in images], 0), torch.FloatTensor(targets)

def test_collate_fn(batch):
    images = batch
    return torch.cat([img.reshape(-1, 1, 448, 448) for img in images], 0)

In [ ]:
train_loader = DataLoader(train_ds, batch_size=20, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_ds, batch_size=20, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=test_collate_fn)

# Yolo Network

In [ ]:
# YoLo
class YoLo_v1(nn.Module):
    def __init__(self, num_classes=20, num_bboxes=2):
        super(YoLo_v1, self).__init__()

        self.feature_size = 7
        self.num_bboxes=num_bboxes
        self.num_classes=num_classes
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=4),
            # nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(192),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=192, out_channels=128, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=2, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),

            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
        )

        self.fc = nn.Sequential(
            Flatten(),
            nn.Linear(in_features=7*7*1024, out_features=4096),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=(feature_size*feature_size*(5 * num_bboxes + num_classes))),
            nn.Softmax()
        )
            
        self.init_weight(self.conv)
        self.init_weight(self.fc)

    def forward(self, x):
        s, b, c = self.feature_size, self.num_bboxes, self.num_classes

        x = self.conv(x)
        x = self.fc(x)

        x = x.view(-1, s, s, (5 * b + c))
        return x

    def init_weight(self, modules):
        for m in modules:
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='leaky_relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

class Squeeze(nn.Module):
    def __init__(self):
        super(Squeeze, self).__init__()

    def forward(self, x):
        return x.squeeze()

class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        return x.view(x.size(0), -1)

## IoU 계산
torch로 계산하냐, numpy로 계산하냐에 따라 차이

In [1]:
# 예시
# compute_iou(pred_xyxy[:, :4], target_xyxy[:, :4]) # [B, 1]
# pred_xyxy : 예측 박스
# target_xyxy : grid sell에서 특징 추출 위치


def compute_iou(bbox1, bbox2):
    """ Compute the IoU (Intersection over Union) of two set of bboxes, each bbox format: [x1, y1, x2, y2].
    Args:
        bbox1: (Tensor) bounding bboxes, sized [N, 4].
        bbox2: (Tensor) bounding bboxes, sized [M, 4].
    Returns:
        (Tensor) IoU, sized [N, M].
    """
    N = bbox1.size(0)
    M = bbox2.size(0)

    # Compute left-top coordinate of the intersections
    lt = torch.max(
        bbox1[:, :2].unsqueeze(1).expand(N, M, 2), # [N, 2] -> [N, 1, 2] -> [N, M, 2]
        bbox2[:, :2].unsqueeze(0).expand(N, M, 2)  # [M, 2] -> [1, M, 2] -> [N, M, 2]
    )
    # Conpute right-bottom coordinate of the intersections
    rb = torch.min(
        bbox1[:, 2:].unsqueeze(1).expand(N, M, 2), # [N, 2] -> [N, 1, 2] -> [N, M, 2]
        bbox2[:, 2:].unsqueeze(0).expand(N, M, 2)  # [M, 2] -> [1, M, 2] -> [N, M, 2]
    )
    # Compute area of the intersections from the coordinates
    wh = rb - lt   # width and height of the intersection, [N, M, 2]
    wh[wh < 0] = 0 # clip at 0
    inter = wh[:, :, 0] * wh[:, :, 1] # [N, M]

    # Compute area of the bboxes
    area1 = (bbox1[:, 2] - bbox1[:, 0]) * (bbox1[:, 3] - bbox1[:, 1]) # [N, ]
    area2 = (bbox2[:, 2] - bbox2[:, 0]) * (bbox2[:, 3] - bbox2[:, 1]) # [M, ]
    area1 = area1.unsqueeze(1).expand_as(inter) # [N, ] -> [N, 1] -> [N, M]
    area2 = area2.unsqueeze(0).expand_as(inter) # [M, ] -> [1, M] -> [N, M]

    # Compute IoU from the areas
    union = area1 + area2 - inter # [N, M, 2]
    iou = inter / union           # [N, M, 2]

    return iou

# Loss function

In [ ]:
# 예시
# loss = loss_fn(output, target).cuda()
# output : yolo network 돌아갔다 옴.

def loss_fn(pred_tensor, target_tensor):
    """ Compute loss for YOLO training.
    Args:
        pred_tensor: (Tensor) predictions, sized [n_batch, S, S, Bx5+C], 5=len([x, y, w, h, conf]).
        target_tensor: (Tensor) targets, sized [n_batch, S, S, Bx5+C].
    Returns:
        (Tensor): loss, sized [1, ].
    """
    
    # TODO: Romove redundant dimensions for some Tensors.
    S = feature_size
    B = num_bboxes
    C = num_classes
    N = 5 * B + C 
    
    lambda_coord = 5    #torch.tensor().cuda()
    lambda_noobj = 0.5  #torch.tensor(0.5).cuda()

    # pred_tensor = output
    batch_size = pred_tensor.size(0)
    coord_mask = target_tensor[:, :, :, 4] > 0  # mask for the cells which contain objects. [n_batch, S, S]
    noobj_mask = target_tensor[:, :, :, 4] == 0 # mask for the cells which do not contain objects. [n_batch, S, S]
    coord_mask = coord_mask.unsqueeze(-1).expand_as(target_tensor) # [n_batch, S, S] -> [n_batch, S, S, N]
    noobj_mask = noobj_mask.unsqueeze(-1).expand_as(target_tensor) # [n_batch, S, S] -> [n_batch, S, S, N]

    coord_pred = pred_tensor[coord_mask].view(-1, N)            # pred tensor on the cells which contain objects. [n_coord, N]
                                                                # n_coord: number of the cells which contain objects.
    bbox_pred = coord_pred[:, :5*B].contiguous().view(-1, 5)    # [n_coord x B, 5=len([x, y, w, h, conf])]
    class_pred = coord_pred[:, 5*B:]                            # [n_coord, C]

    coord_target = target_tensor[coord_mask].view(-1, N)        # target tensor on the cells which contain objects. [n_coord, N]
                                                                # n_coord: number of the cells which contain objects.
    bbox_target = coord_target[:, :5*B].contiguous().view(-1, 5)# [n_coord x B, 5=len([x, y, w, h, conf])]
    class_target = coord_target[:, 5*B:]                        # [n_coord, C]

    # Compute loss for the cells with no object bbox.
    noobj_pred = pred_tensor[noobj_mask].view(-1, N)        # pred tensor on the cells which do not contain objects. [n_noobj, N]
                                                            # n_noobj: number of the cells which do not contain objects.
    noobj_target = target_tensor[noobj_mask].view(-1, N)    # target tensor on the cells which do not contain objects. [n_noobj, N]
                                                            # n_noobj: number of the cells which do not contain objects.
    noobj_conf_mask = torch.cuda.ByteTensor(noobj_pred.size()).fill_(0) # [n_noobj, N]
    
    
    for b in range(B):
        noobj_conf_mask[:, 4 + b*5] = 1 # noobj_conf_mask[:, 4] = 1; noobj_conf_mask[:, 9] = 1
        
    noobj_pred_conf = noobj_pred[noobj_conf_mask]       # [n_noobj, 2=len([conf1, conf2])]
    noobj_target_conf = noobj_target[noobj_conf_mask]   # [n_noobj, 2=len([conf1, conf2])]
    loss_noobj = F.mse_loss(noobj_pred_conf, noobj_target_conf, reduction='sum')

    # Compute loss for the cells with objects.
    coord_response_mask = torch.cuda.ByteTensor(bbox_target.size()).fill_(0)    # [n_coord x B, 5]
    coord_not_response_mask = torch.cuda.ByteTensor(bbox_target.size()).fill_(1)# [n_coord x B, 5]
    bbox_target_iou = torch.zeros(bbox_target.size()).cuda()                    # [n_coord x B, 5], only the last 1=(conf,) is used

    # Choose the predicted bbox having the highest IoU for each target bbox.
    for i in range(0, bbox_target.size(0), B):
        pred = bbox_pred[i:i+B] # predicted bboxes at i-th cell, [B, 5=len([x, y, w, h, conf])]
        pred_xyxy = Variable(torch.FloatTensor(pred.size())) # [B, 5=len([x1, y1, x2, y2, conf])]
        # Because (center_x,center_y)=pred[:, 2] and (w,h)=pred[:,2:4] are normalized for cell-size and image-size respectively,
        # rescale (center_x,center_y) for the image-size to compute IoU correctly.
        pred_xyxy[:,  :2] = pred[:, :2]/float(S) - 0.5 * pred[:, 2:4]
        pred_xyxy[:, 2:4] = pred[:, :2]/float(S) + 0.5 * pred[:, 2:4]

        target = bbox_target[i] # target bbox at i-th cell. Because target boxes contained by each cell are identical in current implementation, enough to extract the first one.
        target = bbox_target[i].view(-1, 5) # target bbox at i-th cell, [1, 5=len([x, y, w, h, conf])]
        target_xyxy = Variable(torch.FloatTensor(target.size())) # [1, 5=len([x1, y1, x2, y2, conf])]
        # Because (center_x,center_y)=target[:, 2] and (w,h)=target[:,2:4] are normalized for cell-size and image-size respectively,
        # rescale (center_x,center_y) for the image-size to compute IoU correctly.
        target_xyxy[:,  :2] = target[:, :2]/float(S) - 0.5 * target[:, 2:4]
        target_xyxy[:, 2:4] = target[:, :2]/float(S) + 0.5 * target[:, 2:4]

        iou = compute_iou(pred_xyxy[:, :4], target_xyxy[:, :4]) # [B, 1]
        max_iou, max_index = iou.max(0)
        max_index = max_index.data.cuda()

        coord_response_mask[i+max_index] = 1
        coord_not_response_mask[i+max_index] = 0

        # "we want the confidence score to equal the intersection over union (IOU) between the predicted box and the ground truth"
        # from the original paper of YOLO.
        bbox_target_iou[i+max_index, torch.LongTensor([4]).cuda()] = (max_iou).data.cuda()
        
        
    bbox_target_iou = Variable(bbox_target_iou).cuda()

    # BBox location/size and objectness loss for the response bboxes.
    bbox_pred_response = bbox_pred[coord_response_mask].view(-1, 5)      # [n_response, 5]
    bbox_target_response = bbox_target[coord_response_mask].view(-1, 5)  # [n_response, 5], only the first 4=(x, y, w, h) are used
    target_iou = bbox_target_iou[coord_response_mask].view(-1, 5)        # [n_response, 5], only the last 1=(conf,) is used
    loss_xy = F.mse_loss(bbox_pred_response[:, :2], bbox_target_response[:, :2], reduction='sum')
    loss_wh = F.mse_loss(torch.sqrt(bbox_pred_response[:, 2:4]), torch.sqrt(bbox_target_response[:, 2:4]), reduction='sum')
    loss_obj = F.mse_loss(bbox_pred_response[:, 4], target_iou[:, 4], reduction='sum')

    # Class probability loss for the cells which contain objects.
    loss_class = F.mse_loss(class_pred, class_target, reduction='sum')

    # Total loss
    loss = lambda_coord * (loss_xy + loss_wh) + loss_obj + lambda_noobj * loss_noobj + loss_class
    loss = loss / float(batch_size)

    return loss

# 학습하기

In [ ]:
yolo = YoLo_v1().cuda()
# criterion = loss_fn().cuda()
init_lr = 0.001
base_lr = 0.01
optimizer = optim.Adam(yolo.parameters(), lr=init_lr, momentum=0.9, weight_decay=0.0005)

def update_lr(optimizer, epoch, step_per, burnin_exp=4.0):
    if epoch in range(50):
        lr = init_lr + (base_lr - init_lr) * math.pow(epoch/(50-1), burnin_exp)
    elif epoch == 50:
        lr = base_lr
    elif epoch == 750:
        lr = 0.001
    elif epoch == 1050:
        lr = 0.0001
    else: return

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
start_time = time.time()
bl = len(train_loader)
history = {'total_loss':[]}


for epoch in range(2):  # loop over the dataset multiple times
    
    tk0 = tqdm(train_loader, total=bl,leave=False)
    t_loss = 0.0
    breaking=False
    
    for step, (image, target) in enumerate(tk0, 0):
        image, target = image.to(device), target.to(device)
        update_lr(optimizer, epoch, float(step) / float(bl - 1))
        output = yolo(image)
        loss = loss_fn(output, target).cuda()

        if math.isnan(loss):
            print(loss)
            breaking = True
            break
            
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        t_loss += loss.item()

        history['total_loss'].append(loss.item())

    if breaking:
        break

    # print statistics
    tqdm.write('[Epoch : %d] total_loss: %.5f Total_elapsed_time: %d 분' %
        (epoch + 1, t_loss / bl, (time.time()-start_time)/60))

print(time.time()-start_time)
print('Finished Training')

# Validation

In [ ]:
def decode(pred_tensor):
    """ Decode tensor into box coordinates, class labels, and probs_detected.
    Args:
        pred_tensor: (tensor) tensor to decode sized [S, S, 5 x B + C], 5=(x, y, w, h, conf)
    Returns:
        boxes: (tensor) [[x1, y1, x2, y2]_obj1, ...]. Normalized from 0.0 to 1.0 w.r.t. image width/height, sized [n_boxes, 4].
        labels: (tensor) class labels for each detected boxe, sized [n_boxes,].
        confidences: (tensor) objectness confidences for each detected box, sized [n_boxes,].
        class_scores: (tensor) scores for most likely class for each detected box, sized [n_boxes,].
    """
    S, B, C = feature_size, num_bboxes, num_classes
    conf_thresh=0.1
    prob_thresh=0.1
    nms_thresh=0.5

    boxes, labels, confidences, class_scores = [], [], [], []

    cell_size = 1.0 / float(S)

    conf = pred_tensor[:, :, 4].unsqueeze(2) # [S, S, 1]
    for b in range(1, B):
        conf = torch.cat((conf, pred_tensor[:, :, 5*b + 4].unsqueeze(2)), 2)
    conf_mask = conf > conf_thresh # [S, S, B]

    # TBM, further optimization may be possible by replacing the following for-loops with tensor operations.
    for i in range(S): # for x-dimension.
        for j in range(S): # for y-dimension.
            class_score, class_label = torch.max(pred_tensor[j, i, 5*B:], 0)

            for b in range(B):
                conf = pred_tensor[j, i, 5*b + 4]
                prob = conf * class_score
                if float(prob) < prob_thresh:
                    continue

                # Compute box corner (x1, y1, x2, y2) from tensor.
                box = pred_tensor[j, i, 5*b : 5*b + 4]
                x0y0_normalized = torch.FloatTensor([i, j]) * cell_size # cell left-top corner. Normalized from 0.0 to 1.0 w.r.t. image width/height.
                xy_normalized = box[:2] * cell_size + x0y0_normalized   # box center. Normalized from 0.0 to 1.0 w.r.t. image width/height.
                wh_normalized = box[2:] # Box width and height. Normalized from 0.0 to 1.0 w.r.t. image width/height.
                box_xyxy = torch.FloatTensor(4) # [4,]
                box_xyxy[:2] = xy_normalized - 0.5 * wh_normalized # left-top corner (x1, y1).
                box_xyxy[2:] = xy_normalized + 0.5 * wh_normalized # right-bottom corner (x2, y2).

                # Append result to the lists.
                boxes.append(box_xyxy)
                labels.append(class_label)
                confidences.append(conf)
                class_scores.append(class_score)

    if len(boxes) > 0:
        boxes = torch.stack(boxes, 0) # [n_boxes, 4]
        labels = torch.stack(labels, 0)             # [n_boxes, ]
        confidences = torch.stack(confidences, 0)   # [n_boxes, ]
        class_scores = torch.stack(class_scores, 0) # [n_boxes, ]
    else:
        # If no box found, return empty tensors.
        boxes = torch.FloatTensor(0, 4)
        labels = torch.LongTensor(0)
        confidences = torch.FloatTensor(0)
        class_scores = torch.FloatTensor(0)

    return boxes, labels, confidences, class_scores